# Tutorial 3 - Memory

## Key Philosophy
- It would be important to learn from past experience and improve the agentic framework - memory is key to that
- You can add to the memory bank of your Agents pre-inference (by collecting from a pool of data prior to running the Agent), or during inference (add on in between running subtasks)

- **Note: Memory will likely be revamped in the future to support vector database insertion and retrieval, for faster runtime**

## Use Memory in Agents
- Agent class takes `memory_bank` as a parameter during initialisation of an `Agent`
- memory_bank: class Dict[Memory]. Stores multiple types of memory for use by the agent. Customise the Memory config within the Memory class.
    - Default: `memory_bank = {'Function': Memory(top_k = 5, mapper = lambda x: x.fn_description, approach = 'retrieve_by_ranker'), llm = self.llm}`
    - Define `memory['Function'].retrieve_fn` in `memory_bank` if you would like to not use OpenAI API. This takes in the task and outputs a list of top k memory items
    - Key: `Function` (Already Implemented Natively) - Does RAG over Task -> Function mapping
    - Can add in more keys that would fit your use case. Retrieves similar items to task/overall plan (if able) for additional context in `get_next_subtasks()` and `use_llm()` function
    - Side Note: RAG can also be done (and may be preferred) as a separate function of the Agent to retrieve more information when needed (so that we do not overload the Agent with information)

## Memory Class
- Retrieves top k memory items based on task 
- Inputs:
    - `memory`: List. Default: Empty List. The list containing the memory items
    - `top_k`: Int. Default: 3. The number of memory list items to retrieve
    - `mapper`: Function. Maps the memory item to another form for comparison by ranker or LLM. Default: `lambda x: x`
        - Example mapping: `lambda x: x.fn_description` (If x is a Class and the string you want to compare for similarity is the fn_description attribute of that class)
    - `approach`: str. Either `retrieve_by_ranker` or `retrieve_by_llm` to retrieve memory items.
        - Ranker is faster and cheaper as it compares via embeddings, but are inferior to LLM-based methods for contextual information
    - `retrieve_fn`: Default: None. Takes in task and outputs top_k similar memories in a list
    - `ranker`: `Ranker`. The Ranker which defines a similarity score between a query and a key. Default: OpenAI `text-embedding-3-small` model. 
        - Can be replaced with a function which returns similarity score from 0 to 1 when given a query and key
        
## Example Use Case
- Helps to reduce number of functions present in LLM context for more accurate generation
```python
output = my_agent.run('Calculate 2**10 * (5 + 1) / 10')
```

`Original Function List: add_numbers, subtract_numbers, add_three_numbers, multiply_numbers, divide_numbers, power_of, GCD_of_two_numbers, modulo_of_numbers, absolute_difference, generate_poem_with_numbers, List_related_words, generate_quote`

`Filtered Function Names: add_three_numbers, multiply_numbers, divide_numbers, power_of, modulo_of_numbers`

# Setup Guide

## Step 1: Install TaskGen

In [1]:
# !pip install taskgen-ai

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from taskgen import *
import os

# this is only if you use OpenAI as your LLM
os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string
- Note: If you do not define your own LLM, OpenAI 'gpt-3.5-turbo' will be used as default

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Positive'

# Use Case 1: Filtering Functions by Task
- TaskGen chooses `top k` (default k: 5) functions according to similarity to subtask
- In addition to `top k` functions, we will also give agent all the compulsory functions
    - `is_compulsory` variable of Function set to `True` means that we will always have it as one of the functions for planning and bypass Function RAG

In [5]:
from typing import List
import math

def sum_numbers(num_list: List[float]) -> float:
    '''Adds all numbers in num_list'''
    return sum(x for x in num_list)

def subtract_numbers(num1: float, num2: float) -> float:
    '''Subtracts num1 from num2'''
    return num1 - num2

def multiply_numbers(num1: float, num2: float) -> float:
    '''Multiplies num1 by num2'''
    return num1 * num2

def divide_numbers(num1: float, num2: float) -> float:
    '''Divides num1 by num2'''
    if num2 == 0:
        return -1
    return num1/num2

def power_operation(num1: float, num2: float) -> float:
    '''Returns num1 to the power of num2 (num1**num2)'''
    return math.pow(num1, num2)

def greatest_common_divisor(num1: int, num2: int) -> int:
    '''Returns greatest common divisor of num1 and num2'''
    return math.gcd(num1, num2)

def modulo(num1: int, num2: int) -> int:
    '''Returns modulo of num1 over num2'''
    return num1%num2

def absolute_difference(num1: int, num2: int) -> int:
    '''Returns absolute difference between num1 and num2'''
    return math.abs(num1-num2)

# Put this to make sum_numbers always appear for any task and bypass Function RAG
sum_numbers = Function(external_fn = sum_numbers, is_compulsory = True)

# This is for internal functions
generate_poem_with_numbers = Function("Generates a poem containing <num1: float> and <num2: float>", output_format = {"Poem": "Poem"}, fn_name = 'generate_poem_with_numbers', llm = llm)
list_related_words = Function("Lists out <num: int> words related to <word: str>", output_format = {"List of words": "List of words, type: list"}, fn_name = 'list_related_words', llm = llm)
generate_quote = Function("Generates a quote about <topic: str>", output_format = {"Quote": "Quote"}, fn_name = 'generate_quote', llm = llm)

In [6]:
my_agent = Agent('Generalist Agent', 
'''Does everything''',
                default_to_llm = False,
                llm = llm).assign_functions([sum_numbers, subtract_numbers, multiply_numbers, 
            divide_numbers, power_operation, greatest_common_divisor, modulo, absolute_difference, 
            generate_poem_with_numbers, list_related_words, generate_quote])

In [7]:
# see the auto-generated names of your functions :)
my_agent.list_functions()

['Name: end_task\nDescription: Passes the final output to the user\nInput: []\nOutput: {}\n',
 "Name: sum_numbers\nDescription: Adds all numbers in <num_list: list[float]>\nInput: ['num_list']\nOutput: {'output_1': 'float'}\n",
 "Name: subtract_numbers\nDescription: Subtracts <num1: float> from <num2: float>\nInput: ['num1', 'num2']\nOutput: {'output_1': 'float'}\n",
 "Name: multiply_numbers\nDescription: Multiplies <num1: float> by <num2: float>\nInput: ['num1', 'num2']\nOutput: {'output_1': 'float'}\n",
 "Name: divide_numbers\nDescription: Divides <num1: float> by <num2: float>\nInput: ['num1', 'num2']\nOutput: {'output_1': 'float'}\n",
 "Name: power_operation\nDescription: Returns <num1: float> to the power of <num2: float> (<num1: float>**<num2: float>)\nInput: ['num1', 'num2']\nOutput: {'output_1': 'float'}\n",
 "Name: greatest_common_divisor\nDescription: Returns greatest common divisor of <num1: int> and <num2: int>\nInput: ['num1', 'num2']\nOutput: {'output_1': 'int'}\n",
 "Nam

In [8]:
# visualise how the Functions are chosen based on task - here you see subtract_numbers appearing at the front
# this does not include the compulsory functions
[f.fn_name for f in my_agent.memory_bank['Function'].retrieve_by_ranker('subtract numbers')]

['subtract_numbers',
 'divide_numbers',
 'multiply_numbers',
 'modulo',
 'generate_poem_with_numbers']

In [9]:
my_agent.reset()
my_agent.run('Evaluate 2+3')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: No subtasks have been completed yet for the assigned task.
Thoughts: To evaluate 2+3, I can use the equipped function sum_numbers by passing the numbers 2 and 3 as input.
Subtask identified: Use the sum_numbers function with input num_list=[2, 3] to add the numbers 2 and 3.
Calling function sum_numbers with parameters {'num_list': [2.0, 3.0]}
> {'output_1': 5.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The sum of 2 and 3 has been successfully calculated using the 'sum_numbers' function.
Thoughts: Since the sum of 2 and 3 has been calculated, the next step is to present this result to the user using the 'end_task' function.
Subtask identified: End Task
Task completed successfully!



[{'output_1': 5.0}]

In [10]:
my_agent.reset()
output = my_agent.run('Evaluate 2**10 * (3+5) / 10')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: No subtasks have been completed yet.
Thoughts: The Assigned Task involves evaluating a mathematical expression. To complete this task, I need to perform the operations in the correct order: first calculate 2 raised to the power of 10, then add 3 and 5, multiply the result by the previous value, and finally divide by 10.
Subtask identified: Calculate 2 raised to the power of 10.
Calling function power_operation with parameters {'num1': 2.0, 'num2': 10.0}
> {'output_1': 1024.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, divide_numbers, power_operation, modulo
Observation: The power_operation function has been successfully used to calculate 2**10.
Thoughts: To complete the remainder of the Assigned Task, we need to perform multiplication of the result by the sum of 3 and 5, and then divide the total by 10.
Subtas

In [11]:
my_agent.reply_user()

819.2


819.2

In [12]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['end_task', 'sum_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power_operation', 'greatest_common_divisor', 'modulo', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote']
Shared Variables: ['agent']
Task: Evaluate 2**10 * (3+5) / 10
Subtasks Completed:
Subtask: power_operation(num1=2.0, num2=10.0)
{'output_1': 1024.0}

Subtask: multiply_numbers(num1=1024.0, num2=8.0)
{'output_1': 8192.0}

Subtask: divide_numbers(num1=8192.0, num2=10.0)
{'output_1': 819.2}

Subtask: sum_numbers(num_list=[3.0, 5.0])
{'output_1': 8.0}

Subtask: sum_numbers(num_list=[1024.0, 8.0])
{'output_1': 1032.0}

Subtask: Evaluate 2**10 * (3+5) / 10
819.2

Is Task Completed: False


# Use Case 2: Adding more context based on task
- You can add additional context as the memory_bank so that `top k` will be added to prompt based on task
- Best not to do this way for large memory banks, consider using a specialised RAG function instead

### Using `memory_bank` for more context
- Here, we have a mapping of nonsense words to numbers
- Based on the subtask, we will augment the system prompt with relevant mappings to aid planning

In [13]:
# first append more context to the agent
my_agent.memory_bank['Word to Numbers'] = Memory([{'Azo': 1}, {'Boneti': 2}, {'Andkh': 3}, {'Bdakf': 4}, {'dafdsk': 5}, 
            {'ldsfn': 6}, {'sdkfn': 7}, {'eri': 8}, {'knewro': 9}, {'mdsnfk': 10}], # some nonsense words
            top_k = 5,  # choose top 5
            mapper = lambda x: list(x.keys())) # we compare with the task using only the first word, e.g. Azo, Boneti, Andkh

In [14]:
my_agent.reset()
output = my_agent.run('Tell me what is Boneti + mdsnfk + Azo')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, modulo, generate_poem_with_numbers, generate_quote
Observation: The Assigned Task involves adding the numerical values of the words "Boneti", "mdsnfk", and "Azo". The numerical values for these words are provided in the Knowledge Reference.
Thoughts: To complete the Assigned Task, we need to convert the words to their numerical values and then sum them up.
Subtask identified: Convert the words "Boneti", "mdsnfk", and "Azo" to their numerical values and then sum them up.
Calling function sum_numbers with parameters {'num_list': [2.0, 10.0, 1.0]}
> {'output_1': 13.0}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, modulo, generate_poem_with_numbers, generate_quote
Observation: One part of the Assigned Task has been completed by summing numbers 2.0, 10.0, and 1.0 to get 13.0.
Thoughts: To complete the remainder of the Assigned Task, we need to convert the words 'Boneti'

In [15]:
my_agent.reply_user()

The task "Tell me what is Boneti + mdsnfk + Azo" cannot be completed based on the information available in the Global Context and Subtasks Completed. No relevant information is present to determine the result of the addition operation between the given terms.


'The task "Tell me what is Boneti + mdsnfk + Azo" cannot be completed based on the information available in the Global Context and Subtasks Completed. No relevant information is present to determine the result of the addition operation between the given terms.'

## Alternative approach of providing additional information via a function
- Sometimes it might be better off to do the additional information providing in a separate function
- This helps us to do more specific augmentation, like doing RAG over documents, or doing rule-based augmentation
- This also helps to reduce the context length of the planner by offloading the augmentation to another function

In [35]:
# delete number meanings additional context memory to showcase the information providing function
if 'Word to Numbers' in my_agent.memory_bank:
    del my_agent.memory_bank['Word to Numbers']

In [36]:
# add in additional information function
# can also do RAG here if needed
def convert_word_to_number(list_of_words: list):
    '''Gets additional information about all unknown words in user_query'''
    word_to_numbers = {'Azo': 1, 'Boneti': 2, 'Andkh': 3, 'Bdakf': 4, 'dafdsk': 5, 
               'ldsfn': 6, 'sdkfn': 7, 'eri': 8, 'knewro': 9, 'mdsnfk': 10}
    
    output_string = ''
    
    list_of_words = str(list_of_words).lower()
    for key, value in word_to_numbers.items():
        if key.lower() in list_of_words.lower():
            output_string += f'{key} equals {value}, '
    return output_string

info_fn = Function('Gets additional information about all unknown words in <list_of_words: List[str]>',
                  output_format = {'Output': 'str'},
                  is_compulsory = True, #makes this function always available for agent
                  external_fn = convert_word_to_number)

info_fn(['Boneti', 'mdsnfk', 'Azo'])

{'Output': 'Azo equals 1, Boneti equals 2, mdsnfk equals 10, '}

In [37]:
# Assign newest function 
my_agent.assign_functions([info_fn])

In [38]:
my_agent.reset()
output = my_agent.run('Tell me what is Boneti + mdsnfk + Azo')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, modulo, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: The Assigned Task involves adding unknown words and performing mathematical operations on them.
Thoughts: Since the task involves unknown words, the first step would be to convert these words to numbers using the 'convert_word_to_number' function. Once the words are converted to numbers, we can proceed with the mathematical operations.
Subtask identified: Convert the words 'Boneti', 'mdsnfk', and 'Azo' to numbers using the 'convert_word_to_number' function.
Calling function convert_word_to_number with parameters {'list_of_words': ['Boneti', 'mdsnfk', 'Azo']}
> {'Output': 'Azo equals 1, Boneti equals 2, mdsnfk equals 10, '}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, modulo, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: The words in the Assigne

In [39]:
my_agent.reply_user()

Azo equals 1, Boneti equals 2, mdsnfk equals 10


'Azo equals 1, Boneti equals 2, mdsnfk equals 10'

# Use Case 3: Adding more context of when to call various functions
- We can augment memory bank with information of what functions to call for certain queries, so that agent knows what to do for some edge cases
- This helps to augment with the system prompt of `get_next_subtask` and `use_llm` with more examples related to the user query

In [68]:
my_agent.memory_bank['Priority Task to Function'] = Memory([
    {'Task': 'Evaluate Azo + eri', 'Function1': {'name': 'convert_word_to_number', 'list_of_words': ['Azo', 'eri']}, 
     'Function2': {'name': 'sum_numbers_in_list', 'num_list': [1, 8]}},
    {'Task': 'Evaluate 5 + 2', 'Function': 'sum_numbers_in_list', 'num_list': [5, 2]},
    {'Task': 'Find out about Boneti and Andkh', 'Function': 'convert_word_to_number', 'list_of_words': ['Boneti', 'Andkh']},
    {'Task': 'What is Andkh?', 'Function': 'convert_word_to_number', 'list_of_words': ['Andkh']},
    {'Task': 'Booyah', 'Function': 'generate_quote', 'topic': 'TaskGen'}
      ], 
    top_k = 3,  # choose top 3
    mapper = lambda x: x['Task']) # we compare with the task using only with the user query

In [69]:
my_agent.reset()
output = my_agent.run('Tell me what is Boneti + mdsnfk + Azo')

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, modulo, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: The Assigned Task involves adding unknown words and performing arithmetic operations on them.
Thoughts: Since the task involves unknown words, the first step would be to convert these words to numbers using the available function convert_word_to_number. Once the words are converted to numbers, the arithmetic operations can be performed accordingly.
Subtask identified: Convert the words "Boneti", "mdsnfk", and "Azo" to numbers.
Calling function convert_word_to_number with parameters {'list_of_words': ['Boneti', 'mdsnfk', 'Azo']}
> {'Output': 'Azo equals 1, Boneti equals 2, mdsnfk equals 10, '}

Filtered Function Names: end_task, sum_numbers, subtract_numbers, multiply_numbers, modulo, generate_poem_with_numbers, generate_quote, convert_word_to_number
Observation: The output for the words "Boneti", "mdsnfk", and "Azo" 

In [70]:
my_agent.reply_user()

Azo equals 1, Boneti equals 2, mdsnfk equals 10


'Azo equals 1, Boneti equals 2, mdsnfk equals 10'

In [71]:
my_agent.reset()
# This actually is not anything special - but because of memory bank, it is mapped to TaskGen
output = my_agent.run('Booyah')

Filtered Function Names: end_task, sum_numbers, multiply_numbers, divide_numbers, power_operation, modulo, generate_quote, convert_word_to_number
Observation: No subtasks have been completed for the Assigned Task 'Booyah'. The Assigned Task 'Booyah' corresponds to the 'generate_quote' function in the Knowledge Reference.
Thoughts: Since the Assigned Task 'Booyah' directly maps to the 'generate_quote' function in the Knowledge Reference, the next step would be to use the 'generate_quote' function to generate a quote about the topic 'TaskGen'.
Subtask identified: Use the 'generate_quote' function with the input topic='TaskGen' to generate a quote related to the topic 'TaskGen'.
Calling function generate_quote with parameters {'topic': 'TaskGen'}
> {'Quote': 'TaskGen is a powerful tool that simplifies the process of generating tasks.'}

Filtered Function Names: end_task, sum_numbers, multiply_numbers, divide_numbers, power_operation, modulo, generate_quote, convert_word_to_number
Observat

In [72]:
my_agent.status()

Agent Name: Generalist Agent
Agent Description: Does everything
Available Functions: ['end_task', 'sum_numbers', 'subtract_numbers', 'multiply_numbers', 'divide_numbers', 'power_operation', 'greatest_common_divisor', 'modulo', 'absolute_difference', 'generate_poem_with_numbers', 'list_related_words', 'generate_quote', 'convert_word_to_number']
Shared Variables: ['agent']
Task: Booyah
Subtasks Completed:
Subtask: generate_quote(topic="TaskGen")
{'Quote': 'TaskGen is a powerful tool that simplifies the process of generating tasks.'}

Is Task Completed: True


# Comparison between embedding-based methods using Ranker and LLM-based similarity
- Pros of embedding-based similarity comparison: Fast and cheap
- Cons of embedding-based similarity comparison: Not as accurate
- If using default `Ranker` function (OpenAI embedding model), automatically stores new embeddings generated in `database` and uses back known embeddings from `database` when possible, potentially helping to save time and costs
- Select the right method for your use case

- (Advanced Exercise) Instead of using cosine similarity using OpenAI Embeddings, create your own `ranking_fn` within `Ranker` that does similarity search the way you want it to

In [26]:
database = {}
memory = Memory(['hello', 'no worries', 'goodbye', 'hurray'], top_k = 1, ranker = Ranker(database = database))
print('Using embeddings', memory.retrieve_by_ranker('Another word for hi'))
print('Using LLM', memory.retrieve_by_llm('Another word for hi'))

Using embeddings ['hello']
Using LLM ['hello']


In [27]:
# Visualise the keys in the database
database.keys()

dict_keys(['hello', 'Another word for hi', 'no worries', 'goodbye', 'hurray'])

In [28]:
print('Using embeddings', memory.retrieve_by_ranker('What to say when leaving'))
print('Using LLM', memory.retrieve_by_llm('What to say when leaving'))

Using embeddings ['goodbye']
Using LLM ['goodbye']


In [29]:
# Visualise the keys in the database
database.keys()

dict_keys(['hello', 'Another word for hi', 'no worries', 'goodbye', 'hurray', 'What to say when leaving'])

# Add memory directly using pdf, docx, csv, xls files
Adding memory elements one by one can be cumbersome, taskgen memory can take filepath as input and it will split the text content inside the file path either using default splitter or user provided splitter.

Example:

`memory = Memory(top_k = 5)`

`memory.add_file(file_path)`


In [30]:
memory = Memory(top_k =1)
memory.add_file(filepath="./react.pdf")

In [31]:
# See the number of chunks for react.pdf
len(memory.memory)

297

In [32]:
#show the first five chunks
memory.memory[:5]

['Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao\x03*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance',
 'ABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning',
 'in an interleaved man

In [33]:
# this takes a while as we need to compare all chunks
memory.retrieve_by_ranker('What is react')

['space and thought-action occurrence format, ReAct works for diverse tasks with distinct action\nspaces and reasoning needs, including but not limited to QA, fact veriﬁcation, text game, and web\nnavigation. C) Performant and robust :ReAct shows strong generalization to new task instances\nwhile learning solely from one to six in-context examples, consistently outperforming baselines with\nonly reasoning or acting across different domains. We also show in Section 3 additional beneﬁts']

In [34]:
# Async version of memory (faster)
asyncMemory = AsyncMemory(top_k = 1)
asyncMemory.add_file(filepath="./react.pdf")
await asyncMemory.retrieve_by_ranker('What is react')

['space and thought-action occurrence format, ReAct works for diverse tasks with distinct action\nspaces and reasoning needs, including but not limited to QA, fact veriﬁcation, text game, and web\nnavigation. C) Performant and robust :ReAct shows strong generalization to new task instances\nwhile learning solely from one to six in-context examples, consistently outperforming baselines with\nonly reasoning or acting across different domains. We also show in Section 3 additional beneﬁts']